<a href="https://colab.research.google.com/github/Mahnazshamissa/Python/blob/main/asigmo_ex3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab exercise 3


Per Group:
1.   Create a teams-meeting where one person opens the colab notebook and shares the screen (preferably a different person than yesterday)
2.   Don't unfreeze the pretrained ResNet50 and train it together with the last layer from scratch on 20 epochs. What's the best validation accuracy and test accuracy you get?
3.   Pretrain the output layer on 20 epochs, then unfreeze all layers and train it for 10 epochs. What's the best validation accuracy and test accuracy you get?
4.   Pretrain the output layer on 20 epochs, then unfreeze only the first 10 layers and train it for 10 epochs. What's the best validation accuracy and test accuracy you get?
5.   Think about why your network performed better or worse compared to the ~94% accuracy of the baseline network
6.   Change the ResNet50 backbone to a ```EfficientNetB4```(group A and C) or ```VGG19``` (group D and E) and train only the output layer. What's the best validation accuracy and test accuracy you get? (don't forget to also change the preprocessing function)
7.   In each group channel answer the questions below

**Warning:** In steps 3 to 5 make sure you overwrite the model and don't continue training on the model from the previous step!


**Questions**
1.  Assuming a ResNet50 training step with a batchsize of 64 takes around 900ms, how long would one training epoch take on ImageNet (1.3 million training images)?
2. How long would it then take to train the ResNet50 for 100 epochs?

## Setup notebook environment

Make sure the Colab environment has a GPU enabled *Edit->Notebook Settings->Hardware accelerator->Choose GPU*

In [ ]:
! nvidia-smi

Download the dataset first

In [ ]:
! wget https://pub.ist.ac.at/~mlechner/datasets/f8d.tar.gz
! tar -xf f8d.tar.gz

Import python libraries

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Define datasets

In [ ]:
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/test", shuffle=False, batch_size=128, image_size=(256, 256),
)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=64,
  image_size=(256, 256),
  validation_split=0.2,
  subset="training",
  seed=1020202,
)
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
  "f8d/training",
  shuffle=True,
  batch_size=128,
  image_size=(256, 256),
  validation_split=0.2,
  subset="validation",
  seed=1020202,
)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

## Step 2 and 3: Define model and objective

In [ ]:
pretrained_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3), include_top=False, weights="imagenet")

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(256,256,3)),
  tf.keras.layers.experimental.preprocessing.RandomCrop(224, 224),
  tf.keras.layers.Lambda(lambda x: tf.keras.applications.resnet.preprocess_input(x)),
  pretrained_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(8, activation="softmax"),
])
pretrained_model.trainable=False
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

## Step 4: Train the last layer

In [ ]:
training_log = model.fit(
  train_dataset, epochs=20, validation_data=valid_dataset
)

In [ ]:
print(f"Best validation accuracy is {100*np.max(training_log.history['val_sparse_categorical_accuracy']):0.2f}%")

## Step 5: Evaluate the model

In [ ]:
model.evaluate(test_dataset)

## Unfreeze the weights of the last 10 layers

In [ ]:
pretrained_model.trainable = True
for l in pretrained_model.layers:
  l.trainable=False
for i in range(10):
  pretrained_model.layers[-i].trainable=True
  
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

In [ ]:
training_log2 = model.fit(
  train_dataset, epochs=10, validation_data=valid_dataset
)

In [ ]:
model.evaluate(test_dataset)